# imports

In [496]:
import gurobipy as gb
import pandas as pd

import numpy as np
np.set_printoptions(linewidth=100)

import random
import os

import pprint
pp = pprint.PrettyPrinter(compact=True)

# generator

In [497]:
class Generator:
    def __init__(self, stops_num, peak_period, passengers_num, bus_trips,\
                tt_disutility, bus_capacity, tt_adj_stops, dwell_time_stop_i, tt_elasticity):
        self.S = stops_num
        self.T = peak_period
        self.P = passengers_num   # (expected demand p^k ?)
        self.B = bus_trips        # (constant frequency f0 ?)

        self.m = tt_disutility
        self.C = bus_capacity
        self.ta = tt_adj_stops
        self.d = dwell_time_stop_i
        self.e = tt_elasticity
        
        self.s = list(range(1,self.S+1))

        self.t = None
        self.c = None
        self.a = None
        self.K = None
        
        self.K_adj = None
        self.K_num = None
        
        self.gamma_k = None
        
        self.t_dict = None
        self.c_dict = None
        self.a_dict = None
        self.gamma_k_dict = None
        
        self.M_dict = None

        self.__start()
        
    def __start(self):
        self.K, self.K_adj, self.K_num, self.t, self.c, self.a, self.gamma_k = self.__gen_multi_matrix()
        self.t_dict, self.c_dict, self.a_dict, self.gamma_k_dict = self.__gen_multi_dict()
        self.M_dict = self.__build_dataframe()

    def __gen_multi_matrix(self):
        K, K_adj, K_num, t, c, a, gamma_k = [], [], [], [], [], [], []
        # K, K_adj, t, c
        for i in range(1,self.S):
            for j in range(i+1, self.S+1):
                K.append((i,j))
                if(j == i+1):
                    K_adj.append((i,j))
                    t.append(self.ta)
                    c.append(0.0)
                else: 
                    t.append(self.ta * (j-i) + self.d * ((j-1)-i))
                    c.append(self.d * ((j-1)-i))
        # a
        for i in t:
            a.append(np.divide(-self.e, i))
        # K_num
        K_num = list(range(len(K)))
        # gamma_k
        for s,d in K:
            gamma_k.append([(i,j) \
                            for i in self.s \
                            for j in self.s \
                            if(i>=s and j>=s and i<j and i<=d and j<=d)])
        return [K, K_adj, K_num, t, c, a, gamma_k]
    
    def __gen_multi_dict(self):
        t_dict, c_dict, a_dict, gamma_k_dict = {}, {}, {}, {}
        t_dict = dict(zip(self.K, self.t))
        c_dict = dict(zip(self.K, self.c))
        a_dict = dict(zip(self.K, self.a))
        gamma_k_dict = dict(zip(self.K, self.gamma_k))
        return [t_dict, c_dict, a_dict, gamma_k_dict]
    
    def __build_dataframe(self):
        M_data = {'K': self.K, 'K_num': self.K_num, 't': self.t, 'c': self.c, 'a': self.a, 'gamma_k': self.gamma_k}
        M_index = pd.MultiIndex.from_tuples(self.K, names = ["sk", "dk"])
        M_df = pd.DataFrame(data = M_data, index = M_index)
        return M_df

| parameters | variables | baseline value |
|:-|:-:|-:|
| wait time disutility weight | $\mu_w$ | $1.0$ |
| bus capacity | $C$ | $80$ (passengers) |
| travel time between adjacent stops | $t_{i,i+1}$ | $1.5$ (minutes) |
| dwell time at stop $i$ | $t^d_i$ | $0.5$ (minutes) |
| travel time elasticity | $e$ | $-0,5$ (minutes) |
| | | |
| total travel time from stop $i$ to stop $j$ on a local service | $t_{ij}$ | $\sum^{j-1}_{l=i}t_{l,l+1}$$+\sum^{j-1}_{l=i+1}t^d_l$ |
| total travel time savings from running express between stops $i$ and $j$ | $c_{ij}$ | $\sum^{j-1}_{l=i+1}t^d_l$ |
| rate of increase in limited-stop service demand per minute of travel time reduction for O-D pair $k=(s^k,d^k)$ | $a^k$ | $\frac{-e}{t_{s^k,d^k}}$ |
| | | |
| number of passengers | $P$ | $3151$ |
| number of bus trips | $B$ | $24$ |
| number of stops | $S$ | $35$ |
| peak period | $T$ | $120$ (minutes) |

# instance generation

In [498]:
# instance = Generator(35, 120, 3151, 24, 1.0, 80, 1.5, 0.5, -0.5)

# ------------------------------------------------------------------

# hint by Professor Rossi
# (L'Aquila --> Lanciano) simulation instance

#         stops_num    ->    S = 3, (L'Aquila --> intermediate stop --> Lanciano)
#       peak_period    ->    T = all day, 1440 minutes
#    passengers_num    ->    P = 50 passengers
#         bus_trips    ->    B = 2 or 3 bus trips
#     tt_disutility    ->    1.0 (default)
#      bus_capacity    ->    50 passengers
#      tt_adj_stops    ->    1.5 (default) 
# dwell_time_stop_i    ->    0.5 (default) 
#     tt_elasticity    ->   -0.5 (default)

instance = Generator(3, 1440, 50, 3, 1.0, 50, 1.5, 0.5, -0.5)
instance.M_dict

K  K_num    t    c         a                   gamma_k
sk dk                                                             
1  2   (1, 2)      0  1.5  0.0  0.333333                  [(1, 2)]
   3   (1, 3)      1  3.5  0.5  0.142857  [(1, 2), (1, 3), (2, 3)]
2  3   (2, 3)      2  1.5  0.0  0.333333                  [(2, 3)]

In [499]:
print("========== PARAMS ==========")
print()
print(instance.S, "\t[num of stops]")
print(instance.T, "\t[peak period (in minutes)]")
print(instance.P, "\t[num. of passengers]")
print(instance.B, "\t[num. of bus trips]")
print()
print(instance.m, "\t[travel time disutility weight]")
print(instance.C, "\t[bus capacity]")
print(instance.ta, "\t[travel time between adjacent stops (i to j)]")
print(instance.d, "\t[dwell time at stop i]")
print(instance.e, "\t[travel time elasticity]")
print()
print("========== t MATRIX ==========")
"""
    1 --- 2                (dwell time = 0.0, travel time = 1.5)
    1 --- 2 --- 3          (dwell time = 0.5, travel time = 1.5 * 2)
    ...
    9 --- 10               (dwell time = 0.0, travel time = 1.5)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.t_dict[(1,2)])
print(instance.t_dict[(1,3)])
print(instance.t_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].t)
print("(1, 3)", instance.M_dict.loc[1,3].t)
print("(2, 3)", instance.M_dict.loc[2,3].t)
print()
print("========== c MATRIX ==========")
"""
    1 --- 2                (dwell time = 0.0)
    1 --- 2 --- 3          (dwell time = 0.5)
    ...
    9 --- 10               (dwell time = 0.0)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.c_dict[(1,2)])
print(instance.c_dict[(1,3)])
print(instance.c_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].c)
print("(1, 3)", instance.M_dict.loc[1,3].c)
print("(2, 3)", instance.M_dict.loc[2,3].c)
print()
print("========== a MATRIX ==========")
"""
    1 --- 2                (-elasticity / travel time = 1.5)
    1 --- 2 --- 3          (-elasticity / travel time = 1.5 * 2)
    ...
    9 --- 10               (-elasticity / travel time = 1.5)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.a_dict[(1,2)])
print(instance.a_dict[(1,3)])
print(instance.a_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].a)
print("(1, 3)", instance.M_dict.loc[1,3].a)
print("(2, 3)", instance.M_dict.loc[2,3].a)


========== PARAMS ==========

3 	[num of stops]
1440 	[peak period (in minutes)]
50 	[num. of passengers]
3 	[num. of bus trips]

1.0 	[travel time disutility weight]
50 	[bus capacity]
1.5 	[travel time between adjacent stops (i to j)]
0.5 	[dwell time at stop i]
-0.5 	[travel time elasticity]

========== t MATRIX ==========

1.5
3.5
1.5

(1, 2) 1.5
(1, 3) 3.5
(2, 3) 1.5

========== c MATRIX ==========

0.0
0.5
0.0

(1, 2) 0.0
(1, 3) 0.5
(2, 3) 0.0

========== a MATRIX ==========

0.3333333333333333
0.14285714285714285
0.3333333333333333

(1, 2) 0.3333333333333333
(1, 3) 0.14285714285714285
(2, 3) 0.3333333333333333


# model/lp init

In [500]:
incr_stop_skip = gb.Model()
incr_stop_skip_lp = "./lp/incr_stop_skip.lp"

if os.path.exists(incr_stop_skip_lp):
    open(incr_stop_skip_lp, "w").close()

# limited-stop service decision variables

In [501]:
""" ========== BETA ========== """
# lb = 0.0 (default)
# up = 1.0
# integrality of beta and gamma can be relaxed (proposition 1) 
# vtype = gb.GRB.BINARY -> gb.GRB.CONTINUOUS (Proposition 1)
Beta = incr_stop_skip.addVars(instance.s, ub = 1.0, \
                              vtype = gb.GRB.CONTINUOUS, name = 'beta')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== GAMMA ========== """
# lb = 0.0 (default)
# up = 1.0
# integrality of beta and gamma can be relaxed (proposition 1) 
# vtype = gb.GRB.BINARY -> gb.GRB.CONTINUOUS (Proposition 1)
Gamma = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                               vtype = gb.GRB.CONTINUOUS, name = 'gamma')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

# passengers decision variables

In [502]:
""" ========== X ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
X = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'x')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== Y ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
Y = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'y')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== Z ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
Z = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'z')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== W ========== """
# lb = 0.0 (default)
# up = constraint (18)
# vtype = GRB.INTEGER
W = incr_stop_skip.addVars(instance.s[:-1], \
                           vtype = gb.GRB.CONTINUOUS, name = 'w')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

# assumptions

In [503]:
""" amount of passengers (assumption 1) """

pk = dict(zip(instance.K, [random.randint(1,instance.P) for k in instance.K]))
print(pk)

""" frequency F constant (assumption 2) """
F = random.randint(1, instance.B-1)
print(F)

{(1, 2): 29, (1, 3): 44, (2, 3): 19}
1


# objective function

$$\max\sum_{k\in K}p^k\sum_{(i,j)\in\Gamma^k}c_{ij}x^k_{ij}\\-\mu_w\sum_{k\in K}p^k(1-\gamma^k)\frac{1}{2}(\frac{T}{f_0-f}-\frac{T}{f_0})\\-\mu_w\sum_{k\in K}p^k z^k\frac{1}{2}(\frac{T}{f}-\frac{T}{f_0})\tag{1}$$

where:
- $\Gamma^k$ denotes a **set of segments** that can be used to serve O-D pair $k\in K$
- mathematically, for an **O-D pair** $k=(s^k,d^k)\in K$, $\Gamma^k$ is given by $\{(i,j)|i,j\in S,s^k\leq i<j\leq d^k\}$

In [504]:
""" (1) expression - w/ M_dict dataframe """

# objexpr = gb.quicksum(pk[i] for i in instance.M_dict.loc[:].K)\
# * gb.quicksum(instance.M_dict.loc[i].c * X[i] \
#             for i in list(k for j in instance.M_dict.loc[:].gamma_k.values.flatten() for k in j))\
# -(instance.m * (gb.quicksum(pk[i] * (1 - Gamma[i]) \
#                             * 1/2 * (instance.T/(instance.B - F) - instance.T/instance.B) \
#                             for i in instance.M_dict.loc[:].K)))\
# -(instance.m * (gb.quicksum(pk[i] * Z[i] * 1/2 * (instance.T/F - instance.T/instance.B) \
#                             for i in instance.M_dict.loc[:].K)))

""" (1) expression - w/ shortcut dicts """

objexpr = gb.quicksum(pk[i] for i in instance.K) \
* gb.quicksum(instance.c_dict[g] * X[g] for i in instance.K for g in instance.gamma_k_dict[i]) \
- instance.m * gb.quicksum(pk[i] * (1 - Gamma[i]) \
                           * 1/2 * (instance.T/(instance.B - F) - instance.T/instance.B) \
                           for i in instance.K) \
- instance.m * gb.quicksum(pk[i] * Z[i] \
                           * 1/2 * (instance.T/F - instance.T/instance.B) \
                           for i in instance.K)
""" (1) """

incr_stop_skip.setObjective(objexpr, gb.GRB.MAXIMIZE)

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

# limited-stop service route contraints (Alpha)

$$\alpha_{ij}\in\{0,1\}, \forall(i,j)\in\{(i,j)|i,j\in S,i<j\}\\\alpha_{s^+,i},\alpha_{i,s^-}\in\{0,1\}, \forall i\in S\tag{5}$$

In [505]:
""" (5) """

K_Alpha = instance.K + [(0, i) for i in instance.s] + [(i, instance.S+1) for i in instance.s]

Alpha = incr_stop_skip.addVars(K_Alpha, vtype = gb.GRB.BINARY, name = 'alpha')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S\setminus\{|S|\}}\alpha_{s^+,i}=1\tag{2}$$

In [506]:
""" (2) """

expr_2 = gb.quicksum(Alpha[0,i] for i in instance.s[:-1]) == 1

incr_stop_skip.addConstr(expr_2, name = '(2)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S:j<i}\alpha_{ji}+\alpha_{s^+,i}=\sum_{j\in S:j>i}\alpha_{ij}+\alpha_{i,s^-}, \forall i\in S\tag{3}$$

In [507]:
""" (3) """

for i in instance.s:
    incr_stop_skip.addConstr(gb.quicksum(Alpha[j,i] for j in instance.s if(j<i)) + \
                             Alpha[0,i] == \
                             gb.quicksum(Alpha[i,j] for j in instance.s if(j>i)) + \
                             Alpha[i, instance.S+1],
                             name = '(3)[' + str(i) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S\setminus\{1\}}\alpha_{i,s^-}=1\tag{4}$$

In [508]:
""" (4) """

expr_4 = gb.quicksum(Alpha[i,instance.S+1] for i in instance.s[1:]) == 1

incr_stop_skip.addConstr(expr_4, name = '(4)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

# limited-stop service route contraints (Beta, Gamma)

$$\beta_i=\sum_{j\in S:j>i}\alpha_{ij}+\alpha_{i,s^-}, \forall i\in S\tag{6}$$

In [509]:
""" (6) """

for i in instance.s:
    incr_stop_skip.addConstr(Beta[i] == gb.quicksum(Alpha[i,j] for j in instance.s if(j>i)) + \
                             Alpha[i, instance.S+1], \
                             name = '(6)[' + str(i) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\gamma^k\leq\beta_{s^k}, \forall k=(s^k,d^k)\in K\tag{7}$$

In [510]:
""" (7) """

expr_7 = (Gamma[(i,j)] <= Beta[i] for i,j in instance.K)

incr_stop_skip.addConstrs(expr_7, name = '(7)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\gamma^k\leq\beta_{d^k}, \forall k=(s^k,d^k)\in K\tag{8}$$

In [511]:
""" (8) """

expr_8 = (Gamma[(i,j)] <= Beta[j] for i,j in instance.K)

incr_stop_skip.addConstrs(expr_8, name = '(8)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\beta_i\in\{0,1\}, \forall i\in S\tag{9}$$
$$\gamma^k\in\{0,1\}, \forall k\in K\tag{10}$$

In [512]:
""" (9)(10) """

""" already satisfied, decision variables Beta and Gamma can be relaxed (Proposition 1) """

' already satisfied, decision variables Beta and Gamma can be relaxed (Proposition 1) '

# passengers flow contraints

$$x^k_{ij}\leq\alpha_{ij}, \forall k\in K, (i,j)\in\Gamma^k\tag{11}$$

In [513]:
""" (11) """

""" ? ERROR ? """

# incr_stop_skip.addConstrs((X[i,j] <= Alpha[i,j] for i,j in instance.K), \
#                           name = '(11)')

# incr_stop_skip.update()
# incr_stop_skip.write(incr_stop_skip_lp)

' ? ERROR ? '

$$y^k+\sum_{j\in S:s^k<j\leq d^k}x^k_{s^k,j}=1, \forall k=(s^k,d^k)\in K\tag{12}$$

In [514]:
""" (12) """

for i,j in instance.K:
    incr_stop_skip.addConstr(Y[i,j] + \
                             gb.quicksum(X[i,z] for z in instance.s if(z>i and z<=j)) == 1,\
                             name = '(12)[' + str(i) + ',' + str(j) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{j\in S:s^k\leq j<i}x^k_{ji}-\sum_{j\in S:i<j\leq d^k}x^k_{ij}=0, \forall k=(s^k,d^k)\in K, i\in\{i\in S|s^k<i<d^k\}\tag{13}$$

In [515]:
""" (13) """

for z,x in instance.K:
    incr_stop_skip.addConstr(gb.quicksum(X[j,i] \
                                         for i in instance.s if(i>z and i<x) 
                                         for j in instance.s if(j>=z and j<i)) - \
                             gb.quicksum(X[i,j] \
                                         for i in instance.s if(i>z and i<x) \
                                         for j in instance.s if(j>i and j<=x)) == 0, \
                             name = '(13)[' + str(z) + ',' + str(x) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$0\leq x^k_{ij}\leq 1, \forall k\in K, (i,j)\in\Gamma^k\tag{14}$$
$$0\leq y^k\leq 1, \forall k\in K\tag{15}$$

In [516]:
""" (14)(15) """

""" already satisfied, decision variables X and Y can be gb.GRB.CONTINUOUS, ub = 1.0, lb = 0.0 """

' already satisfied, decision variables X and Y can be gb.GRB.CONTINUOUS, ub = 1.0, lb = 0.0 '

# capacity contraints [extra]

$$w_1=\sum_{k\in K,s^k=1}p^ky^k\tag{16}$$

In [517]:
""" (16) """

expr_16 = W[1] == gb.quicksum(pk[i] * Y[i] for i in instance.K if(i[0] == 1))

incr_stop_skip.addConstr(expr_16, name = '(16)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$w_i=\sum_{k\in K,s^k\leq i}p^ky^k-\sum_{k\in K,d^k\leq i}p^ky^k, \forall i\in S\setminus\{1,|S|\}\tag{17}$$

In [518]:
""" (17) """

for i in instance.s[1:-1]:
    incr_stop_skip.addConstr(W[i] == \
                             gb.quicksum(pk[s,d] * Y[s,d] \
                                         for s,d in instance.K if(s<=i)) - \
                             gb.quicksum(pk[s,d] * Y[s,d] \
                                         for s,d in instance.K if(d<=i)), \
                             name = '(17)[' + str(i) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$0\leq w_i \leq(f_0-f)C, \forall i\in S\setminus\{|S|\}\tag{18}$$

In [519]:
""" (18) """

expr_18_lb = (0 <= W[i] for i in instance.s[:-1])
expr_18_ub = (W[i] <= ((instance.B - F) * instance.C) for i in instance.s[:-1])

incr_stop_skip.addConstrs(expr_18_lb, name = '(18_lb)')
incr_stop_skip.addConstrs(expr_18_ub, name = '(18_ub)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{k\in K:\Gamma^k\ni(i,j)}p^kx^k_{ij}\leq(fC)\alpha_{ij}, \forall(i,j)\in\{(i,j)|i,j\in S,i<j\}\tag{19}$$

In [520]:
""" (19) """

""" ? ERROR ? """

# for i,j in instance.K:
#     incr_stop_skip.addConstr(gb.quicksum(pk[i,j] * X[z,x] \
#                                          for z,x in instance.gamma_k_dict[i,j]) <= \
#                              (F * instance.C) * Alpha[i,j], \
#                              name = '(19)[' + str(i) + ',' + str(j) + ']')
    
# incr_stop_skip.update()
# incr_stop_skip.write(incr_stop_skip_lp)

' ? ERROR ? '

$$f\in\{1,2,\ldots,f_0-1\}\tag{20}$$

In [521]:
""" (20) """

""" already satisfied, static setting of variable F (Decomposition) """

' already satisfied, static setting of variable F (Decomposition) '

# demand split contraints [extra] $a^k$

$$\sum_{j\in S:s^k<j\leq d^k}x^k_{s^k,j}\leq\frac{f}{f_0}+a^k(\sum_{(i,j)\in\Gamma^k}\alpha_{ij}c_{ij}), \forall k\in K\tag{21}$$

where:
- $a^k$'s are constants representing the incremental proportion of passengers preferring the limited-stop service per minute of travel time reduction

In [522]:
""" (21) """

""" ? REDUNDANT ? """

# for s,d in instance.K:
#     incr_stop_skip.addConstr(gb.quicksum(X[s,j] \
#                                          for j in instance.s if(j>s and j<=d)) <= \
#                              (F / instance.B) + instance.a_dict[s,d] * \
#                              gb.quicksum(Alpha[z,x] * instance.c_dict[z,x] \
#                                          for z,x in instance.gamma_k_dict[s,d]), \
#                              name = '(21)[' + str(s) + ',' + str(d) + ']')
    
# incr_stop_skip.update()
# incr_stop_skip.write(incr_stop_skip_lp)

' ? REDUNDANT ? '

# demand split contraints [extra] $z^k$

$$z^k\geq\sum_{j\in S:s^k<j\leq d^k}x^k_{s^k,j}-\frac{f}{f_0}, \forall k\in K\tag{22}$$

In [523]:
""" (22) """

""" using both (21) and (22) the solver fails """

for s,d in instance.K:
    incr_stop_skip.addConstr(Z[s,d] <= \
                             gb.quicksum(X[s,j] \
                                         for j in instance.s if(j>s and j<=d)) - \
                             (F/instance.B), \
                             name = '(22)[' + str(s) + ',' + str(d) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$0\leq z^k\leq 1, \forall k\in K\tag{23}$$

In [524]:
""" (23) """

"""" already satisfied, decision variables Z can be gb.GRB.CONTINUOUS, ub = 1.0, lb = 0.0 """

'" already satisfied, decision variables Z can be gb.GRB.CONTINUOUS, ub = 1.0, lb = 0.0 '

In [525]:
""" optimization hook """

incr_stop_skip.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 29 rows, 26 columns and 63 nonzeros
Model fingerprint: 0xfc2e1a37
Variable types: 17 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 29 rows and 26 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: 30.6667 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.066666666667e+01, best bound 3.066666666667e+01, gap 0.0000%


In [526]:
""" optimization results """
incr_stop_skip.ObjVal, incr_stop_skip.Runtime

(30.66666666666606, 0.020923852920532227)